In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer= WordNetLemmatizer()
import json
import pickle
import random
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
words =[]
classes =[]
documents=[]
ignore_words=['?','!','.','-']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenization
        w=nltk.word_tokenize(pattern)
        print('Token is :{}'.format((w)))
        words.extend(w)
        documents.append((w,intent['tag']))
    #     add the tag to classes list
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

Token is :['hii']
Token is :['hello']
Token is :['hey']
Token is :['what', "'s", 'up', '?']
Token is :['hello', 'IGDTUW']
Token is :['how', 'many', 'departments', 'are', 'there', 'in', 'IGDTUW', '?']
Token is :['No', '.', 'of', 'Depts', '.', '?']
Token is :['Dept', '.', 'count', '?']
Token is :['Departments']
Token is :['how', 'many', 'courses', 'are', 'there', 'in', 'PhD', 'at', 'IGDTUW', '?']
Token is :['No', '.', 'of', 'PhD', 'courses', '?']
Token is :['PhD', 'Courses', 'count', '?']
Token is :['PhD', 'Courses']
Token is :['what', 'branches', 'are', 'there', 'in', 'PhD', '?']
Token is :['PhD', 'branches', '?']
Token is :['branches', 'in', 'PhD']
Token is :['PhD', 'branch', 'names']
Token is :['what', 'courses', 'are', 'there', 'in', 'UG', '?']
Token is :['Courses', 'in', 'UG']
Token is :['UG', 'Courses']
Token is :['what', 'branches', 'are', 'there', 'in', 'Btech', '?']
Token is :['BTech', 'branches', '?']
Token is :['branches', 'in', 'BTech']
Token is :['BTech', 'branch', 'names']


In [4]:
documents

[(['hii'], 'greetings'),
 (['hello'], 'greetings'),
 (['hey'], 'greetings'),
 (['what', "'s", 'up', '?'], 'greetings'),
 (['hello', 'IGDTUW'], 'greetings'),
 (['how', 'many', 'departments', 'are', 'there', 'in', 'IGDTUW', '?'], 'dept'),
 (['No', '.', 'of', 'Depts', '.', '?'], 'dept'),
 (['Dept', '.', 'count', '?'], 'dept'),
 (['Departments'], 'dept'),
 (['how', 'many', 'courses', 'are', 'there', 'in', 'PhD', 'at', 'IGDTUW', '?'],
  'phd'),
 (['No', '.', 'of', 'PhD', 'courses', '?'], 'phd'),
 (['PhD', 'Courses', 'count', '?'], 'phd'),
 (['PhD', 'Courses'], 'phd'),
 (['what', 'branches', 'are', 'there', 'in', 'PhD', '?'], 'phd-branch'),
 (['PhD', 'branches', '?'], 'phd-branch'),
 (['branches', 'in', 'PhD'], 'phd-branch'),
 (['PhD', 'branch', 'names'], 'phd-branch'),
 (['what', 'courses', 'are', 'there', 'in', 'UG', '?'], 'ug-course'),
 (['Courses', 'in', 'UG'], 'ug-course'),
 (['UG', 'Courses'], 'ug-course'),
 (['what', 'branches', 'are', 'there', 'in', 'Btech', '?'], 'btech-branch'),
 (

In [5]:
words=[lemmatizer.lemmatize(w.lower())for w in words if w not in ignore_words]
words=list(set(words))
classes=list(set(classes))
print(words)
print(classes)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

['btech/mca', 'major', 'mtech', 'festival', 'recent', 'where', 'year', 'internship', 'event', 'department', 'visit', 'tech-fest', 'held', 'cse/it', 'no', 'statistic', 'both', 'last', 'hey', 'of', 'course', 'up', 'and', 'by', 'here', 'full', 'for', 'specification', 'ignite', 'pg', 'taarangana', 'how', 'that', 'technical', 'hello', 'dual-degree', 'hii', 'it', 'phd', 'training', 'name', 'the', 'placement', 'are', 'departemnt', 'club', 'at', 'igdtuw', 'tpo', 'which', 'activity', 'find', 'mca', 'is', 'co-curricular', 'mean', 'mae', 'sport', 'career', 'coming', 'campus', 'innerve', 'in', 'branch', 'form', 'ant', 'who', 'conatct', 'btech', 'company', 'cse/ece/it', 'extra-curricular', 'degree', 'many', 'technical-societies', 'dept', 'there', 'detail', 'society', 'cse', 'depts', 'graduate', 'what', 'dual', 'about', 'option', 'count', 'visiting', "'s", 'to', 'ug', 'you', 'tnp', 'officer', 'do', 'contact']
['activities', 'btech-branch', 'career-option', 'events', 'tpo-contact', 'tpo', 'ug-course'

In [7]:
training = []
output_empty = [0]*len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    print('Current Pattern Words: {}'.format(pattern_words))
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    print('Current Bag: {}'.format(bag))
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    print("Current Output: {}".format(output_row))
    
    training.append([bag, output_row])

Current Pattern Words: ['hii']
Current Bag: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Current Output: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Current Pattern Words: ['hello']
Current Bag: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Current Output: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Current Pattern Words: ['hey']
Current Bag: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [8]:
print("Training: {}".format(training))

Training: [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]], [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [9]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])
print('X: {}'.format(train_x))
print('Y: {}'.format(train_y))

X: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

<ipython-input-9-38f09c84ede1>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [15]:
model = keras.Sequential()
model.add(layers.Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


c:\users\swati basu\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [16]:
mfit = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
22/22 [==============================] - 0s 855us/step - loss: 3.3436 - accuracy: 0.0556
Epoch 2/200
22/22 [==============================] - 0s 807us/step - loss: 3.2783 - accuracy: 0.1019
Epoch 3/200
22/22 [==============================] - 0s 807us/step - loss: 3.1056 - accuracy: 0.1296
Epoch 4/200
22/22 [==============================] - 0s 2ms/step - loss: 3.0330 - accuracy: 0.0926
Epoch 5/200
22/22 [==============================] - 0s 760us/step - loss: 2.8660 - accuracy: 0.1759
Epoch 6/200
22/22 [==============================] - 0s 760us/step - loss: 2.7038 - accuracy: 0.2222
Epoch 7/200
22/22 [==============================] - 0s 760us/step - loss: 2.5829 - accuracy: 0.2593
Epoch 8/200
22/22 [==============================] - 0s 760us/step - loss: 2.4952 - accuracy: 0.3148
Epoch 9/200
22/22 [==============================] - 0s 712us/step - loss: 2.4556 - accuracy: 0.3333
Epoch 10/200
22/22 [==============================] - 0s 618us/step - loss: 2.2772 - accuracy

22/22 [==============================] - 0s 617us/step - loss: 0.1571 - accuracy: 0.9259
Epoch 161/200
22/22 [==============================] - 0s 570us/step - loss: 0.1810 - accuracy: 0.9259
Epoch 162/200
22/22 [==============================] - 0s 522us/step - loss: 0.2314 - accuracy: 0.9074
Epoch 163/200
22/22 [==============================] - 0s 570us/step - loss: 0.1854 - accuracy: 0.9167
Epoch 164/200
22/22 [==============================] - 0s 617us/step - loss: 0.1791 - accuracy: 0.9259
Epoch 165/200
22/22 [==============================] - 0s 617us/step - loss: 0.1420 - accuracy: 0.9352
Epoch 166/200
22/22 [==============================] - 0s 617us/step - loss: 0.2076 - accuracy: 0.9074
Epoch 167/200
22/22 [==============================] - 0s 617us/step - loss: 0.2068 - accuracy: 0.8704
Epoch 168/200
22/22 [==============================] - 0s 617us/step - loss: 0.1891 - accuracy: 0.9352
Epoch 169/200
22/22 [==============================] - 0s 570us/step - loss: 0.2602 - a

In [17]:
model.save('chatbot_model.h5', mfit)